In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from audiomentations import *
import warnings
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from datetime import datetime
import json

ModuleNotFoundError: No module named 'audiomentations'

In [ ]:
data_dir = "/media/mountHDD3/data_storage/biomedical_data/ecg_data/SPH"
print(os.listdir(data_dir))

In [ ]:
main_df = pd.read_csv(data_dir + "/data_df.csv")
main_df.shape

In [ ]:
single_main_df = main_df[main_df["Second_label"].isnull()]
single_main_df.shape

In [ ]:
# single_main_df.to_csv(data_dir + "/single_label.csv")

In [ ]:
single_fns = main_df["File name"].values.tolist()
single_mat_paths = [data_dir + f"/records/{x}.h5" for x in single_fns]

In [ ]:
ratio = [0.9, 0.1]

train_index = int(len(single_mat_paths)*ratio[0])

train_mat_paths = single_mat_paths[:train_index]
valid_mat_paths = single_mat_paths[train_index:]

In [ ]:
sample_sig = torch.randn(1, 12, 32)
conv_test = nn.Conv1d(12, 12, 3, 1, 1)
print(conv_test(sample_sig).shape)

In [ ]:
# len_lst = [loadmat(x)['ECG'][0][0][2].shape[1] for x in single_mat_paths]
# print(f"MAX: {max(len_lst)}")
# print(f"MIN: {min(len_lst)}")
# print(f"AVG: {sum(len_lst)/len(len_lst)}")

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, channel_num):
        super(BasicBlock, self).__init__()
        self.conv_block1 = nn.Sequential(
			nn.Conv1d(channel_num, channel_num, 3, padding=1),
			nn.BatchNorm1d(channel_num),
			nn.LeakyReLU(0.2),
		)
        self.conv_block2 = nn.Sequential(
			nn.Conv1d(channel_num, channel_num, 3, padding=1),
			nn.BatchNorm1d(channel_num),
		)
        self.relu = nn.LeakyReLU(0.2)
        torch.nn.init.kaiming_normal_(self.conv_block1[0].weight)
        torch.nn.init.kaiming_normal_(self.conv_block2[0].weight)
        
    def forward(self, x):
        residual = x
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x + residual
        out = self.relu(x)
        return out

In [ ]:
test_basic_block = BasicBlock(2)
sample_sig = torch.randn(1, 2, 32)
print(test_basic_block(sample_sig).shape)

In [ ]:
class ResNet(nn.Module):
    def __init__(self, in_channels = 12, type = 18, num_classes = 34):
        super(ResNet, self).__init__()
        self.struc_dict = {
            18: {
                "num_channels" : [64, 128, 256, 512],
                "counts" : [2, 2, 2, 2]
            }
        }
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2)
        torch.nn.init.kaiming_normal_(self.conv1.weight)
        self.max1 = nn.MaxPool1d(kernel_size=3, stride=2)
        self.main = nn.Sequential()
        for idx, struc in enumerate(
            zip(
                self.struc_dict[type]["num_channels"], 
                self.struc_dict[type]["counts"]
            )
        ):
            num_channel, cnt = struc
            for i in range(cnt):
                self.main.add_module(f"conv{idx+1}_{i}", BasicBlock(num_channel))
            if idx < len(self.struc_dict[type]["num_channels"]) - 1:
                self.main.add_module(f"ext_{idx}", nn.Conv1d(num_channel, self.struc_dict[type]["num_channels"][idx+1], 3, 1))
                self.main.add_module(f"extbn_{idx}", nn.BatchNorm1d(self.struc_dict[type]["num_channels"][idx+1]))
                                     
        self.avg = torch.nn.AdaptiveAvgPool1d((1))
        self.lin = nn.Linear(self.struc_dict[type]["num_channels"][-1], num_classes)
        torch.nn.init.kaiming_normal_(self.lin.weight)
    def forward(self, x):
        x = self.conv1(x)
        x = self.max1(x)
        x = self.main(x)
        x = self.avg(x)
        x = x.reshape(x.shape[0], -1)
        x = self.lin(x)
        return x

In [ ]:
model = ResNet()
sample_sig = torch.randn(1, 12, 3000)
model(sample_sig).shape

In [ ]:
class ECG(Dataset):
    def __init__(self, data_paths, label_df, train):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        self.train = train

    def __getitem__(self, idx):
        augment = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.01, p=0.2),
            AddGaussianSNR(min_snr_db=3.0, max_snr_db=12.0, p=0.2),
            Gain(min_gain_db = -10.0, max_gain_db = 10.0, p =0.2),
            GainTransition(min_gain_db = -15.0, max_gain_db = 15.0, 
                           min_duration = 50, max_duration = 200, duration_unit = "samples", 
                           p = 0.2),
            TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2),
            PitchShift(min_semitones=-4, max_semitones=4, p=0.2),
            Shift(min_fraction=-0.5, max_fraction=0.5, p=0.2),
        ])
        
        data_path = self.data_paths[idx]        

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["File name"] == filename]["New Label"].values.item()
          
        tensor_lst = []
        for i in range(12):    
            data_path = self.data_paths[idx]
            a = h5py.File(data_path, 'r')
            data_h5 = a['ecg']
            data = np.array(data_h5)
            clip_data = data[i][500:]
            desired_length = 2500
            if self.train == True:
                pad_length = max(desired_length - len(clip_data), 0)
                if pad_length == 0:
                    padded_augment = clip_data[:desired_length]
                else:
                    padded_augment = np.pad(clip_data, (0, pad_length), mode='constant')

            else:
                augmented_samples = augment(samples=clip_data, sample_rate=500)
                pad_length = max(desired_length - len(augmented_samples), 0)
                if pad_length == 0:
                    padded_augment = augmented_samples[:desired_length]
                else:
                    padded_augment = np.pad(clip_data, (0, pad_length), mode='constant')

            augment_signal_tensor = torch.tensor(padded_augment).float()
            tensor_lst.append(augment_signal_tensor)
            
        n = torch.stack(tensor_lst, 0).float()

        return n, label

    def __len__(self):
        return len(self.data_paths)

In [ ]:
# check_ds = ECG(data_paths=single_mat_paths, label_df=single_main_df)
# sample, lbl = check_ds[0]
# print(sample.shape, lbl)

In [ ]:
# model(sample.unsqueeze(dim=0)).shape

In [ ]:
train_ds = ECG(train_mat_paths, main_df, train = True)
valid_ds = ECG(valid_mat_paths, main_df, train = False)

print(len(train_ds))
print(len(valid_ds))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 0)
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

print(len(traindl))
print(len(validdl))

In [ ]:
epoch = 150
lr = 0.0005

model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for e in range(epoch):
    model.train()
    print(f"Epoch: {e}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = loss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
    
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")
    
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_sig = valid_sig.to(device)
            valid_label = valid_label.to(device)
            
            pred = model(valid_sig)
            loss = loss_fn(pred, valid_label)
            
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
    
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        
        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")